---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    
    
    return spam_data.target.sum()/spam_data.target.count()*100

answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    
    vect = CountVectorizer().fit(X_train)
    tokens = vect.get_feature_names()
    return max(tokens, key=len)

answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [5]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    
    vect = CountVectorizer().fit(X_train)
    X_vector = vect.transform(X_train)
    model = MultinomialNB(alpha=0.1).fit(X_vector, y_train)
    y_predict = model.predict(vect.transform(X_test))
    
    return roc_auc_score(y_test, y_predict)

answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    
    vect = TfidfVectorizer().fit(X_train)
    X_vect = vect.transform(X_train)
    
    X_feature = pd.DataFrame(X_vect.max(0).toarray()[0], columns=['tfidf'])
    X_feature['name'] = vect.get_feature_names()
    X_feature = X_feature.sort_values(['tfidf', 'name'])
    small = X_feature.iloc[:20]
    small = small.set_index('name')
    small = small['tfidf']
    small.index.name = None
    small.name = None
    X_feature = X_feature.sort_values(['tfidf', 'name'], ascending=[False, True])
    large = X_feature.iloc[:20]
    large = large.set_index('name')
    large = large['tfidf']
    large.index.name = None
    large.name = None
    return small, large

answer_four()

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 afternoons      0.091250
 approaching     0.091250
 dtype: float64, 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 tick         0.980166
 blank        0.932702
 dty

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
def answer_five():
    
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_vect = vect.transform(X_train)
    
    model = MultinomialNB(alpha=0.1).fit(X_vect, y_train)
    y_predict = model.predict(vect.transform(X_test))
    
    return roc_auc_score(y_test, y_predict)

answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [8]:
def answer_six():
    
    copy = spam_data.copy()
    copy['char'] = copy['text'].str.len()
    char = copy.groupby('target').mean()['char']
    
    return tuple(x for x in char)

answer_six()

(71.023626943005183, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [9]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [10]:
from sklearn.svm import SVC

def answer_seven():
    
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_vec = vect.transform(X_train)
    X_vec = add_feature(X_vec, [len(x) for x in X_train])
    
    model = SVC(C=10000).fit(X_vec, y_train)
    X_test_vec = add_feature(vect.transform(X_test), [len(x) for x in X_test])
    y_predict = model.predict(X_test_vec)
    
    return roc_auc_score(y_test, y_predict)

answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [11]:
def answer_eight():
    
    copy = spam_data.copy()
    copy['digits'] = copy['text'].apply(lambda doc: len([char for char in doc if char.isdigit()]))
    return tuple(x for x in copy.groupby('target').mean()['digits'])

answer_eight()

(0.29927461139896372, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [12]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    
    vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    length = [len(x) for x in X_train]
    digits = [len([char for char in doc if char.isdigit()]) for doc in X_train]
    
    X_vec = vect.transform(X_train)
    X_vec = add_feature(X_vec, length)
    X_vec = add_feature(X_vec, digits)
    
    model = LogisticRegression(C=100).fit(X_vec, y_train)
    
    test_length = [len(x) for x in X_test]
    test_digits = [len([char for char in doc if char.isdigit()]) for doc in X_test]
    
    X_test_vec = vect.transform(X_test)
    X_test_vec = add_feature(X_test_vec, test_length)
    X_test_vec = add_feature(X_test_vec, test_digits)
    
    y_predict = model.predict(X_test_vec)
    
    return roc_auc_score(y_test, y_predict)

answer_nine()

0.96533283533945646

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [13]:
def answer_ten():
    
    copy = spam_data.copy()
    copy['non-word'] = copy['text'].str.findall('\W').str.len()
    return tuple(x for x in copy.groupby('target').mean()['non-word'])

answer_ten()

(17.291813471502589, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [14]:
def answer_eleven():
    
    import re
    
    vect = CountVectorizer(min_df=5, ngram_range=(2,5), analyzer='char_wb').fit(X_train)
    
    def transform(vectorizer, X):
        length = [len(x) for x in X]
        digits = [len([char for char in doc if char.isdigit()]) for doc in X]
        nonword = [len(re.findall('\W', doc)) for doc in X]
        
        X_vec = vectorizer.transform(X)
        X_vec = add_feature(X_vec, length)
        X_vec = add_feature(X_vec, digits)
        X_vec = add_feature(X_vec, nonword)
        
        return X_vec
        
    X_train_vec = transform(vect, X_train)
    model = LogisticRegression(C=100).fit(X_train_vec, y_train)
    
    X_test_vec = transform(vect, X_test)
    y_predict = model.predict(X_test_vec)
    score = roc_auc_score(y_test, y_predict)
    
    coef = list(model.coef_[0])
    coef_ind = model.coef_[0].argsort()
    
    coef.sort()
    small = coef[:10]
    large = coef[-1:-11:-1]
    
    return score, small, large

answer_eleven()

(0.97885931107074342,
 [-0.86975346259415875,
  -0.86088296352174276,
  -0.67698172297661741,
  -0.66700363899346971,
  -0.61490070391470086,
  -0.57958954863764711,
  -0.53507444581451002,
  -0.50576580840067331,
  -0.49850657095575379,
  -0.49096047527002135],
 [1.2122304448365826,
  0.59777832370761941,
  0.54148300240059377,
  0.53876611978762901,
  0.52148696634166225,
  0.52035142621300001,
  0.51786804549367671,
  0.51609606976459066,
  0.50867081207580411,
  0.50264408611612776])